In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib 
import os

C:\Users\Luis\AppData\Roaming\Python\Python38\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
os.chdir('C:/Users/Luis/Box/R15 Sensor Preprocessing and Analysis/JPCS Special Issue') #set directory
raw = pd.read_csv('Full_Summary_Features_clean_2-24-2024.csv') #filtered summary data from madison

In [3]:
for col in raw.columns: 
    print(col) #i want to see all the columns so i can specifically pick the features i actually want
    #we are going to make a dataset containing just restrict alongside predictors aka physio
    #then we are going to make another dataset containing just urge to restrict alongside physio
    #we don't want to include any other EMA in our train/test data for each analysis

Unnamed..0
Time
HR_Mean
HR_Minimum
HR_Maximum
HR_Stdev
HR_RMS
HR_MAD
HR_MAV
HR_Median
HR_P25
HR_P75
EDA_Mean
EDA_Minimum
EDA_Maximum
EDA_Stdev
EDA_RMS
EDA_MAD
EDA_MAV
EDA_Median
EDA_P25
EDA_P75
TEMP_Mean
TEMP_Minimum
TEMP_Maximum
TEMP_Stdev
TEMP_RMS
TEMP_MAD
TEMP_MAV
TEMP_Median
TEMP_P25
TEMP_P75
meanCenteredEDA_Mean
meanCenteredEDA_Minimum
meanCenteredEDA_Maximum
meanCenteredEDA_Stdev
meanCenteredEDA_RMS
meanCenteredEDA_MAD
meanCenteredEDA_MAV
meanCenteredEDA_Median
meanCenteredEDA_P25
meanCenteredEDA_P75
meanCenteredHR_Mean
meanCenteredHR_Minimum
meanCenteredHR_Maximum
meanCenteredHR_Stdev
meanCenteredHR_RMS
meanCenteredHR_MAD
meanCenteredHR_MAV
meanCenteredHR_Median
meanCenteredHR_P25
meanCenteredHR_P75
meanCenteredTEMP_Mean
meanCenteredTEMP_Minimum
meanCenteredTEMP_Maximum
meanCenteredTEMP_Stdev
meanCenteredTEMP_RMS
meanCenteredTEMP_MAD
meanCenteredTEMP_MAV
meanCenteredTEMP_Median
meanCenteredTEMP_P25
meanCenteredTEMP_P75
ethica_time
lag
tdif
cumsumT
ethica_time_utc
dayvar
beepvar


In [4]:
#random seed. I picked 6 because I like 6 and i always pick 6. this is so we get the same results every time
np.random.seed(6)

In [5]:
#creating the dataset for predicting urge_restrict from physio alone
urgeData = raw.filter(['meanCenteredEDA_Mean', 'meanCenteredEDA_Minimum', 'meanCenteredEDA_Maximum', 'meanCenteredEDA_Stdev', 
                       'meanCenteredEDA_RMS', 'meanCenteredEDA_MAD', 'meanCenteredEDA_MAV', 'meanCenteredEDA_Median', 
                       'meanCenteredEDA_P25', 'meanCenteredEDA_P75', 'meanCenteredHR_Mean', 'meanCenteredHR_Minimum', 
                       'meanCenteredHR_Maximum', 'meanCenteredHR_Stdev', 'meanCenteredHR_RMS', 'meanCenteredHR_MAD', 
                       'meanCenteredHR_MAV', 'meanCenteredHR_Median', 'meanCenteredHR_P25', 'meanCenteredHR_P75', 
                       'meanCenteredTEMP_Mean', 'meanCenteredTEMP_Minimum', 'meanCenteredTEMP_Maximum', 'meanCenteredTEMP_Stdev', 
                       'meanCenteredTEMP_RMS', 'meanCenteredTEMP_MAD', 'meanCenteredTEMP_MAV', 'meanCenteredTEMP_Median', 
                       'meanCenteredTEMP_P25', 'meanCenteredTEMP_P75', 'urge_restrict'])




In [6]:
#creating the dataset for predicting restrict from physio alone
restrictData = raw.filter(['meanCenteredEDA_Mean', 'meanCenteredEDA_Minimum', 'meanCenteredEDA_Maximum', 'meanCenteredEDA_Stdev', 
                       'meanCenteredEDA_RMS', 'meanCenteredEDA_MAD', 'meanCenteredEDA_MAV', 'meanCenteredEDA_Median', 
                       'meanCenteredEDA_P25', 'meanCenteredEDA_P75', 'meanCenteredHR_Mean', 'meanCenteredHR_Minimum', 
                       'meanCenteredHR_Maximum', 'meanCenteredHR_Stdev', 'meanCenteredHR_RMS', 'meanCenteredHR_MAD', 
                       'meanCenteredHR_MAV', 'meanCenteredHR_Median', 'meanCenteredHR_P25', 'meanCenteredHR_P75', 
                       'meanCenteredTEMP_Mean', 'meanCenteredTEMP_Minimum', 'meanCenteredTEMP_Maximum', 'meanCenteredTEMP_Stdev', 
                       'meanCenteredTEMP_RMS', 'meanCenteredTEMP_MAD', 'meanCenteredTEMP_MAV', 'meanCenteredTEMP_Median', 
                       'meanCenteredTEMP_P25', 'meanCenteredTEMP_P75', 'restrict'])



In [7]:
urgeData.dropna(subset=["urge_restrict"], inplace = True) #remove any cases where EMA is missing
np.any(np.isnan(urgeData['urge_restrict'])) #check that NAs exist - if false, we are good

restrictData.dropna(subset=["restrict"], inplace = True) #remove any cases where EMA is missing
np.any(np.isnan(restrictData['restrict'])) #check that NAs exist - if false, we are good

False

In [8]:
#we want to randomly assort the rows in these dataframes now since they're clustered by participant
restrictDataShuffled = restrictData.sample(frac = 1) #shuffle restriction dataset
urgeDataShuffled = urgeData.sample(frac = 1) #shuffle urge dataset

In [9]:
#now what i want to do is reserve a portion of the data to test after the fact - the remaining data we'll do k-fold cross validation
#we're going to split the data as follows: 
#80% of the data is going to be the train/test data for the model
#the remaining 20% will be held and we will use the trained/tested model to see how closely it predicts the 20% holdover set

from sklearn.model_selection import train_test_split

urgeX = urgeDataShuffled.drop(columns = {'urge_restrict'}) #drop urge_restrict from dataframe for the splitting
urgey = urgeDataShuffled.filter(['urge_restrict']) #make a dataframe containing only urge_restrict variable

restX = restrictDataShuffled.drop(columns = {'restrict'})
resty = restrictDataShuffled.filter(['restrict'])

In [10]:
urgeX_train, urgeX_test, urgey_train, urgey_test = train_test_split(urgeX, urgey, random_state=6,test_size=0.20) 
restX_train, restX_test, resty_train, resty_test = train_test_split(restX, resty, random_state=6,test_size=0.20) 
#we're sticking with 6 for random_state
#the test sets are what we are going to validate our models on later

In [11]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import ElasticNet

#we're going to start exclusively with the urge data for now, we'll circle back to the restrict data later 

urgeMod = ElasticNet() #we're starting with default parameters - alpha=1.0, l1_ratio=0.5
urgecv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=6) #i still like 6, we're doing a 10-fold cross-validation repeating 3 times


In [12]:
from sklearn.model_selection import cross_validate
from numpy import mean
from numpy import std
from numpy import absolute

def cross_validation(model, _X, _y, _cv):
    _scoring = ['neg_mean_absolute_error', 'neg_mean_squared_error', 'r2']
    results = cross_validate(estimator = model, X = _X, y = _y, cv = _cv, scoring = _scoring, return_train_score = True)
    #print(results)
    return {#"MAE scores": absolute(results['train_neg_mean_absolute_error']),
            "Mean MAE": absolute(results['train_neg_mean_absolute_error']).mean(),
            #"MSE scores": absolute(results['train_neg_mean_squared_error']),
            "Mean MSE": absolute(results['train_neg_mean_squared_error']).mean(),
            #"R^2 scores": absolute(results['train_r2']),
            "Mean R^2": absolute(results['train_r2']).mean()} #make a function for all performance metrics

In [13]:
urgecv_result = cross_validation(urgeMod, urgeX_train, urgey_train, urgecv)
print(urgecv_result) #well we're on average ~26.5 points off of predicting urge to restrict

{'Mean MAE': 26.47874485549623, 'Mean MSE': 946.6085695917993, 'Mean R^2': 0.03777392875523218}


In [14]:
#might as well try restrict now 

restMod = ElasticNet()
restcv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=6)
restcv_result = cross_validation(restMod, restX_train, resty_train, restcv)
print(restcv_result) #we're on average ~29.5 points off of predicting restriction

{'Mean MAE': 29.522097506654028, 'Mean MSE': 1101.295243658522, 'Mean R^2': 0.04689058977877744}
